# Cricket Match Data Exploration & Quality Validation

This notebook provides comprehensive data exploration and validation before ML model development.

## Contents
1. Setup & Database Connection
2. Data Quality Checks
3. Match-Level Analysis
4. Player Performance Analysis
5. Ball-by-Ball Data Validation
6. ELO System Validation
7. Feature Engineering Preview
8. Data Quality Report

---

## 1. Setup & Database Connection


In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime

from config import DATABASE_PATH, MIN_MATCH_DATE

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

print(f"Database path: {DATABASE_PATH}")
print(f"Min match date filter: {MIN_MATCH_DATE}")


In [ ]:
# Connect to database
conn = sqlite3.connect(DATABASE_PATH)
conn.row_factory = sqlite3.Row

# Helper function to run queries
def query_df(sql, params=None):
    return pd.read_sql_query(sql, conn, params=params)


## 2. Data Quality Checks


In [ ]:
# Database overview - record counts
tables = ['teams', 'players', 'venues', 'matches', 'innings', 'deliveries', 
          'player_match_stats', 'team_elo_history', 'player_elo_history',
          'team_current_elo', 'player_current_elo']

print("=" * 50)
print("DATABASE OVERVIEW")
print("=" * 50)

counts = {}
for table in tables:
    try:
        result = query_df(f"SELECT COUNT(*) as count FROM {table}")
        counts[table] = result['count'].iloc[0]
    except:
        counts[table] = 0

for table, count in counts.items():
    print(f"{table:25} {count:>12,} rows")


In [ ]:
# Match breakdown by format and gender
match_breakdown = query_df("""
    SELECT 
        match_type,
        gender,
        COUNT(*) as matches,
        MIN(date) as first_match,
        MAX(date) as last_match
    FROM matches
    GROUP BY match_type, gender
    ORDER BY match_type, gender
""")

print("\nMATCH BREAKDOWN BY FORMAT & GENDER")
print("=" * 50)
print(match_breakdown.to_string(index=False))


In [ ]:
# Missing values analysis
print("\nMISSING VALUES CHECK")
print("=" * 50)

# Check matches table
matches_nulls = query_df("""
    SELECT 
        COUNT(*) as total,
        SUM(CASE WHEN winner_id IS NULL THEN 1 ELSE 0 END) as no_winner,
        SUM(CASE WHEN toss_winner_id IS NULL THEN 1 ELSE 0 END) as no_toss,
        SUM(CASE WHEN venue_id IS NULL THEN 1 ELSE 0 END) as no_venue,
        SUM(CASE WHEN player_of_match_id IS NULL THEN 1 ELSE 0 END) as no_pom
    FROM matches
""")

print("\nMatches table:")
total = matches_nulls['total'].iloc[0]
print(f"  Total matches: {total:,}")
print(f"  No winner (ties/no result): {matches_nulls['no_winner'].iloc[0]:,} ({matches_nulls['no_winner'].iloc[0]/total*100:.1f}%)")
print(f"  No toss info: {matches_nulls['no_toss'].iloc[0]:,}")
print(f"  No venue: {matches_nulls['no_venue'].iloc[0]:,}")
print(f"  No player of match: {matches_nulls['no_pom'].iloc[0]:,}")


In [ ]:
# Player coverage per match
player_coverage = query_df("""
    SELECT 
        AVG(player_count) as avg_players,
        MIN(player_count) as min_players,
        MAX(player_count) as max_players,
        SUM(CASE WHEN player_count < 18 THEN 1 ELSE 0 END) as incomplete_matches
    FROM (
        SELECT match_id, COUNT(*) as player_count
        FROM player_match_stats
        GROUP BY match_id
    )
""")

print("\nPLAYER DATA COVERAGE")
print("=" * 50)
print(f"Average players per match: {player_coverage['avg_players'].iloc[0]:.1f} (expected: 22)")
print(f"Min players in a match: {player_coverage['min_players'].iloc[0]}")
print(f"Max players in a match: {player_coverage['max_players'].iloc[0]}")
print(f"Matches with < 18 players: {player_coverage['incomplete_matches'].iloc[0]} (potentially incomplete)")


## 3. Match-Level Analysis


In [ ]:
# Load matches data
matches_df = query_df("""
    SELECT 
        m.*,
        t1.name as team1_name,
        t2.name as team2_name,
        w.name as winner_name,
        tw.name as toss_winner_name,
        v.name as venue_name,
        v.city as venue_city
    FROM matches m
    LEFT JOIN teams t1 ON m.team1_id = t1.team_id
    LEFT JOIN teams t2 ON m.team2_id = t2.team_id
    LEFT JOIN teams w ON m.winner_id = w.team_id
    LEFT JOIN teams tw ON m.toss_winner_id = tw.team_id
    LEFT JOIN venues v ON m.venue_id = v.venue_id
""")

matches_df['date'] = pd.to_datetime(matches_df['date'])
matches_df['year'] = matches_df['date'].dt.year
matches_df['month'] = matches_df['date'].dt.month

print(f"Total matches: {len(matches_df):,}")
print(f"Date range: {matches_df['date'].min()} to {matches_df['date'].max()}")
matches_df.head()


In [ ]:
# Match type distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart for match types
type_counts = matches_df['match_type'].value_counts()
axes[0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
axes[0].set_title('Match Type Distribution')

# Matches per year by type
yearly_matches = matches_df.groupby(['year', 'match_type']).size().unstack(fill_value=0)
yearly_matches.plot(kind='bar', ax=axes[1])
axes[1].set_title('Matches per Year by Type')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Number of Matches')

plt.tight_layout()
plt.show()


In [ ]:
# Toss decision impact
toss_df = matches_df[matches_df['winner_id'].notna()].copy()
toss_df['toss_winner_won'] = toss_df['toss_winner_id'] == toss_df['winner_id']

toss_win_rate = toss_df.groupby(['match_type', 'toss_decision'])['toss_winner_won'].mean() * 100

print("\nWin Rate When Winning Toss (%)")
print(toss_win_rate.unstack())


## 4. Score & Player Analysis


In [ ]:
# Load innings data
innings_df = query_df("""
    SELECT 
        i.*,
        m.match_type,
        m.date
    FROM innings i
    JOIN matches m ON i.match_id = m.match_id
""")

# Score distributions by match type
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

t20_scores = innings_df[innings_df['match_type'] == 'T20']['total_runs']
odi_scores = innings_df[innings_df['match_type'] == 'ODI']['total_runs']

axes[0].hist(t20_scores, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_title('T20 Score Distribution')
axes[0].set_xlabel('Runs')
axes[0].axvline(t20_scores.mean(), color='red', linestyle='--', label=f'Mean: {t20_scores.mean():.0f}')
axes[0].legend()

axes[1].hist(odi_scores, bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('ODI Score Distribution')
axes[1].set_xlabel('Runs')
axes[1].axvline(odi_scores.mean(), color='red', linestyle='--', label=f'Mean: {odi_scores.mean():.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nT20 Score Statistics: Mean={t20_scores.mean():.1f}, Std={t20_scores.std():.1f}")
print(f"ODI Score Statistics: Mean={odi_scores.mean():.1f}, Std={odi_scores.std():.1f}")


## 5. Top Performers Validation


In [ ]:
# Top run scorers (T20 Men) - validate against known cricket stars
print("TOP 15 RUN SCORERS - T20 INTERNATIONALS (MEN)")
print("=" * 70)

top_batters_t20m = query_df("""
    SELECT 
        p.name,
        SUM(pms.runs_scored) as total_runs,
        SUM(pms.balls_faced) as total_balls,
        COUNT(*) as matches,
        SUM(pms.fours_hit) as fours,
        SUM(pms.sixes_hit) as sixes,
        ROUND(SUM(pms.runs_scored) * 100.0 / NULLIF(SUM(pms.balls_faced), 0), 1) as strike_rate
    FROM player_match_stats pms
    JOIN players p ON pms.player_id = p.player_id
    JOIN matches m ON pms.match_id = m.match_id
    WHERE m.match_type = 'T20' AND m.gender = 'male'
    GROUP BY p.player_id
    HAVING SUM(pms.balls_faced) >= 100
    ORDER BY total_runs DESC
    LIMIT 15
""")

print(top_batters_t20m.to_string(index=False))

print("\n✓ Check: Do these names match known T20I run scorers like Babar Azam, Rizwan, SKY?")


## 6. ELO System Validation


In [ ]:
# Current team rankings by format and gender
print("CURRENT TEAM ELO RANKINGS")
print("=" * 70)

for fmt in ['T20', 'ODI']:
    for gen in ['male', 'female']:
        col = f'elo_{fmt.lower()}_{gen}'
        rankings = query_df(f"""
            SELECT t.name, ROUND(e.{col}) as elo
            FROM team_current_elo e
            JOIN teams t ON e.team_id = t.team_id
            WHERE e.{col} > 1500
            ORDER BY e.{col} DESC
            LIMIT 10
        """)
        
        print(f"\n{fmt} {gen.upper()} - Top 10:")
        for i, row in rankings.iterrows():
            print(f"  {i+1:2}. {row['name']:25} {int(row['elo'])}")


## 7. Data Quality Report


In [ ]:
# Generate final data quality report
print("=" * 70)
print("DATA QUALITY REPORT")
print("=" * 70)

# Summary statistics
total_matches = counts.get('matches', 0)
total_deliveries = counts.get('deliveries', 0)
total_players = counts.get('players', 0)

print(f"\n1. DATA VOLUME")
print(f"   Total matches: {total_matches:,}")
print(f"   Total deliveries: {total_deliveries:,}")
print(f"   Total players: {total_players:,}")
print(f"   Player-match records: {counts.get('player_match_stats', 0):,}")

print(f"\n2. DATA COMPLETENESS")
incomplete = player_coverage['incomplete_matches'].iloc[0] if 'incomplete_matches' in player_coverage else 0
completeness_pct = (1 - incomplete/total_matches) * 100 if total_matches > 0 else 0
print(f"   Matches with full player data: {completeness_pct:.1f}%")
print(f"   Matches with partial data: {int(incomplete)} (filter these for ML)")

print(f"\n3. ELO SYSTEM")
print(f"   Team ELO records: {counts.get('team_elo_history', 0):,}")
print(f"   Player ELO records: {counts.get('player_elo_history', 0):,}")
print(f"   Separate tracking: T20M, T20F, ODIM, ODIF ✓")

print(f"\n4. RECOMMENDATIONS")
print(f"   - Filter matches with < 18 players for ML training")
print(f"   - Use registry_id for player matching across formats")
print(f"   - Consider re-downloading recent data for completeness")

print(f"\n5. GO/NO-GO ASSESSMENT")
if completeness_pct > 90 and total_matches > 1000:
    print(f"   ✓ DATA QUALITY: GOOD - Ready for ML development")
elif completeness_pct > 80:
    print(f"   ⚠ DATA QUALITY: ACCEPTABLE - Proceed with caution")
else:
    print(f"   ✗ DATA QUALITY: POOR - Address issues before ML")

print("\n" + "=" * 70)


In [ ]:
# Close connection
conn.close()
print("Database connection closed.")
